In [1]:
import aqueduct as aq
from aqueduct.constants.enums import ArtifactType

In [2]:
client = aq.Client(api_key="V3OJFILC64TRW2D9M0GQ8ZHUYXNB5ASP", aqueduct_address="http://3.130.165.53:8080")

# client.list_resources()
aq.global_config({'engine':'databricks_resource', 'lazy':True})

In [3]:
client.list_resources()

{'Demo': {
     "Id": "a7fc7d3b-f12f-4099-8e17-bb2a0b868270",
     "Name": "Demo",
     "Service": "SQLite",
     "Connected On": "2023-05-23 00:24:13",
     "Status": "succeeded"
 },
 'Aqueduct Server': {
     "Id": "ba04f0ec-0300-4e0f-b530-9c73ed7025c7",
     "Name": "Aqueduct Server",
     "Service": "Aqueduct",
     "Connected On": "2023-05-23 00:24:13",
     "Status": "succeeded"
 },
 'Filesystem': {
     "Id": "9d7974ff-2347-4501-a57a-fae45b2954e9",
     "Name": "Filesystem",
     "Service": "Filesystem",
     "Connected On": "2023-05-23 00:24:13",
     "Status": "succeeded"
 },
 'gcs_resource': {
     "Id": "09b532e1-1fd0-4706-9b9e-29724cfb6f7b",
     "Name": "gcs_resource",
     "Service": "GCS",
     "Connected On": "2023-05-23 00:29:31",
     "Status": "succeeded"
 },
 'k8s_resource': {
     "Id": "7b161baf-e7b8-463a-81db-a6f7bc23aef0",
     "Name": "k8s_resource",
     "Service": "Kubernetes",
     "Connected On": "2023-05-23 00:39:42",
     "Status": "succeeded"
 },
 'snowf

In [19]:

snowflake_warehouse = client.resource('snowflake_resource')
hotel_reviews = snowflake_warehouse.sql('SELECT * FROM large_hotel_reviews;')
# hotel_reviews.get()

In [27]:
@aq.op(requirements=[])
def dummy(original_df):
    from pyspark.sql.functions import monotonically_increasing_id
    from pyspark.sql.functions import rand
    import math
    
    original_row_count = original_df.count()
    num_trillion_rows = 10000000000
    num_partitions = int(math.ceil(num_trillion_rows / original_row_count))

    # Step 2: Repartition the DataFrame
    replicated_df = original_df.repartition(num_partitions)

    # Step 3: Persist the DataFrame
    replicated_df.persist()

    # Step 4: Duplicate the rows
    while replicated_df.count() < num_trillion_rows:
        replicated_df = replicated_df.union(replicated_df)

    print(replicated_df.count())

    return replicated_df

generated_df = dummy(hotel_reviews)

In [28]:
snowflake_warehouse.save(generated_df, table_name="large_hotel_reviews", update_mode="replace")


client.publish_flow(
    "Creating_Large_Dataset",
    "repartition hotel_reviews to create big dataset",
    artifacts=[generated_df],
)

Url:  http://3.130.165.53:8080/workflow/1939d5ec-2cda-4921-9356-ad4a6ce066cd
